In [27]:
sc

In [28]:
sc.applicationId

u'application_1522648856070_1091'

In [29]:
import sys
sys.path.append("/usr/lib/python2.7/site-packages")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import col, unix_timestamp,abs, log, from_unixtime, avg
from pyspark.sql.functions import count, sum, desc, date_format, lit, concat
from pyspark.sql import functions as F

In [30]:
# For displaying multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [31]:
sqlContext.sql("use bmtc")

# Get the route map
route_map_df = sqlContext.sql("select route_id,start_bus_stop_id,end_bus_stop_id,\
                                      distance,time_to_travel,bus_stop_order,status \
                               from route_map")

# Get the route_point
route_point_df = sqlContext.sql("select route_id, route_order, bus_stop_id from route_point")

bus_stop_df = sqlContext.sql("select bus_stop_id, bus_stop_name from bus_stop")

# Join the bus stop ID with lat,long
route_point_joined_df = route_point_df.join(bus_stop_df,\
                                            ["bus_stop_id"],\
                                            "left_outer")

form_four_df = sqlContext.sql("select form_four_id,form_four_name,schedule_number_id,\
                                      schedule_number_name,no_of_trips,start_time,\
                                      route_id,route_number,toll_zone,\
                                      area_limit,total_km,total_dead_km,\
                                      actual_km,total_running_time,total_break_time,\
                                      total_steering_time,spread_over_hours,ot_hours \
                               from form_four")

schedule_df = sqlContext.sql('select * from schedule')

schedule_df = schedule_df.select("schedule_id","schedule_number","depot_id","route_id","schedule_type")

schedule_details_df = sqlContext.sql('select * from schedule_details')

schedule_details_df = schedule_details_df.select(col("form_four_id").alias("schedule_no"),"schedule_number","trip_number","trip_type", "start_time", "end_time",
                                                 col("route_number_id").alias("route_id"))

DataFrame[]

In [32]:
sqlContext.sql("use bmtc_eta_default")

DataFrame[]

In [33]:
sqlContext.sql("show tables").show(truncate = False)

+---------------------------+-----------+
|tableName                  |isTemporary|
+---------------------------+-----------+
|vts_365r                   |false      |
|vts_365r_distance          |false      |
|vts_365r_distance_1        |false      |
|vts_365r_filtered          |false      |
|vts_bus_stop_min_dist_apr17|false      |
|vts_bus_stop_min_dist_feb17|false      |
|vts_bus_stop_min_dist_jan17|false      |
|vts_bus_stop_min_dist_mar17|false      |
|vts_jun2017                |false      |
|vts_volvo_2017             |false      |
|vts_volvo_oct2017          |false      |
|vts_waybill_volvo_apr2017  |false      |
|vts_waybill_volvo_feb2017  |false      |
|vts_waybill_volvo_jan2017  |false      |
|vts_waybill_volvo_jun2017  |false      |
|vts_waybill_volvo_mar2017  |false      |
|vts_waybill_volvo_may2017  |false      |
+---------------------------+-----------+



In [89]:
sqlContext.sql("use bmtc_eta_default")
vts_window_df = sqlContext.sql("select * from vts_bus_stop_min_dist_apr17")
vts_window_df = vts_window_df.filter(year(vts_window_df.etm_start_time) == 2017)
vts_window_count = vts_window_df.count()
vts_window_count

DataFrame[]

1966803

In [90]:
vts_window_df.show(1)

+--------+-----------+-------------+--------------------+---------+---------+----------+----------+-----------+----------------+------------+-----------+----------+--------+--------+--------------------+--------------------+-----------+-----------+--------------+----------------+-----------------+----+--------+----------+
|route_id|         id|vts_device_id|            ist_date|      lat|longitude| ist_epoch|   duty_dt|schedule_no|   schedule_name|service_type|trip_number|start_time|end_time|route_no|      etm_start_time|        etm_end_time|bus_stop_id|route_order| bus_stop_name|latitude_current|longitude_current|dist|min_dist|  min_time|
+--------+-----------+-------------+--------------------+---------+---------+----------+----------+-----------+----------------+------------+-----------+----------+--------+--------+--------------------+--------------------+-----------+-----------+--------------+----------------+-----------------+----+--------+----------+
|   29518|15218702893|    15

In [91]:
vts_frame_df = vts_window_df.select("duty_dt", "vts_device_id", "schedule_no", "trip_number", 
                                    "route_id","start_time", "end_time").distinct()
vts_frame_df.show(10)

+----------+-------------+-----------+-----------+--------+----------+--------+
|   duty_dt|vts_device_id|schedule_no|trip_number|route_id|start_time|end_time|
+----------+-------------+-----------+-----------+--------+----------+--------+
|2017-04-01|    150220662|      22872|          1|   23478|  05:30:00|07:10:00|
|2017-04-03|    150219597|      38291|          5|   32611|  19:10:00|20:40:00|
|2017-04-27|    150814426|      21081|          1|   29017|  05:10:00|06:35:00|
|2017-04-29|    150221620|      20312|          5|   29742|  08:50:00|10:30:00|
|2017-04-29|    150222184|      25468|          4|   34851|  10:25:00|11:55:00|
|2017-04-29|    150812093|      24404|          3|   13342|  17:00:00|18:30:00|
|2017-04-10|    150220811|      24444|          5|   32612|  11:55:00|13:30:00|
|2017-04-10|    150220850|      22947|          3|   34851|  15:10:00|16:40:00|
|2017-04-10|    150221286|      12284|          4|   32611|  15:05:00|16:35:00|
|2017-04-11|    150218344|      33323|  

In [92]:
# No of trips made by Volvo buses for the month
vts_frame_count = vts_frame_df.count()
vts_frame_count

70129

In [93]:
vts_frame_all_stops_df = vts_frame_df.join(route_point_joined_df, 
                                           ["route_id"],
                                           "inner")

vts_frame_all_stops_count = vts_frame_all_stops_df.count()
vts_frame_all_stops_count

2966298

In [94]:
vts_frame_all_stops_df.show()

+--------+----------+-------------+-----------+-----------+----------+--------+-----------+-----------+--------------------+
|route_id|   duty_dt|vts_device_id|schedule_no|trip_number|start_time|end_time|bus_stop_id|route_order|       bus_stop_name|
+--------+----------+-------------+-----------+-----------+----------+--------+-----------+-----------+--------------------+
|   13289|2017-04-19|    150220325|      22863|          3|  07:50:00|09:00:00|        372|          1|         Bommasandra|
|   13289|2017-04-19|    150220325|      22863|          3|  07:50:00|09:00:00|        788|          2|           Hebbagodi|
|   13289|2017-04-19|    150220325|      22863|          3|  07:50:00|09:00:00|       1318|          3|         Huskur Gate|
|   13289|2017-04-19|    150220325|      22863|          3|  07:50:00|09:00:00|        370|          4|         Veerasandra|
|   13289|2017-04-19|    150220325|      22863|          3|  07:50:00|09:00:00|       6723|          5|     Electronic City|


In [95]:
vts_frame_all_stops_join_df = vts_frame_all_stops_df.join(vts_window_df, 
                                           ["duty_dt","vts_device_id","schedule_no","route_id",
                                            "trip_number","start_time","end_time", "route_order",
                                            "bus_stop_id", "bus_stop_name"],
                                           "left_outer")

vts_frame_all_stops_join_count = vts_frame_all_stops_join_df.count()
vts_frame_all_stops_join_count

2972514

In [96]:
vts_frame_all_stops_join_df.show(1)

+----------+-------------+-----------+--------+-----------+----------+--------+-----------+-----------+-------------+-----------+--------------------+---------+---------+----------+----------------+------------+--------+--------------------+--------------------+----------------+-----------------+----+--------+----------+
|   duty_dt|vts_device_id|schedule_no|route_id|trip_number|start_time|end_time|route_order|bus_stop_id|bus_stop_name|         id|            ist_date|      lat|longitude| ist_epoch|   schedule_name|service_type|route_no|      etm_start_time|        etm_end_time|latitude_current|longitude_current|dist|min_dist|  min_time|
+----------+-------------+-----------+--------+-----------+----------+--------+-----------+-----------+-------------+-----------+--------------------+---------+---------+----------+----------------+------------+--------+--------------------+--------------------+----------------+-----------------+----+--------+----------+
|2017-04-01|    150218005|     

In [97]:
# vts_window_select_df.groupBy("duty_dt","vts_device_id","schedule_no","route_id",
#                                             "trip_number","start_time","end_time","bus_stop_id",
#                                             "route_order","bus_stop_name").count().\
#                      orderBy("count", ascending=False).show(3)

In [98]:
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window

In [99]:
window = Window().partitionBy("duty_dt","vts_device_id","schedule_no","route_id",
                                            "trip_number","start_time","end_time").orderBy(col("route_order"))
vts_frame_all_stops_lagged_df = vts_frame_all_stops_join_df.select("*", lag("ist_epoch").over(window).alias("lagged_epoch"))
vts_frame_all_stops_join_count = vts_frame_all_stops_join_df.count()
vts_frame_all_stops_join_count

2972514

In [100]:
vts_frame_all_stops_lagged_df.filter(col('trip_number') == 3).show(1)

+----------+-------------+-----------+--------+-----------+----------+--------+-----------+-----------+------------------+-----------+--------------------+---------+---------+----------+------------------+------------+--------+--------------------+--------------------+----------------+-----------------+----+--------+----------+------------+
|   duty_dt|vts_device_id|schedule_no|route_id|trip_number|start_time|end_time|route_order|bus_stop_id|     bus_stop_name|         id|            ist_date|      lat|longitude| ist_epoch|     schedule_name|service_type|route_no|      etm_start_time|        etm_end_time|latitude_current|longitude_current|dist|min_dist|  min_time|lagged_epoch|
+----------+-------------+-----------+--------+-----------+----------+--------+-----------+-----------+------------------+-----------+--------------------+---------+---------+----------+------------------+------------+--------+--------------------+--------------------+----------------+-----------------+----+-----

In [101]:
vts_frame_all_stops_lagged_df.filter((col("duty_dt") == "2017-04-01") &
                                    (col("vts_device_id") == 150222596) &
                                     (col("schedule_no") == 23841) &
                                     (col("route_id") == 30485) &
                                     (col("trip_number") == 3) &
                                     (col("start_time") == "09:05:00") &
                                     (col("end_time") == "10:35:00")).\
                                select("duty_dt","vts_device_id","schedule_no","route_id",
                                            "trip_number","start_time","end_time", "route_order", "bus_stop_name",
                                       "ist_epoch", "lagged_epoch").show(50)

+----------+-------------+-----------+--------+-----------+----------+--------+-----------+--------------------+----------+------------+
|   duty_dt|vts_device_id|schedule_no|route_id|trip_number|start_time|end_time|route_order|       bus_stop_name| ist_epoch|lagged_epoch|
+----------+-------------+-----------+--------+-----------+----------+--------+-----------+--------------------+----------+------------+
|2017-04-01|    150222596|      23841|   30485|          3|  09:05:00|10:35:00|          1|  Central Silk Board|1491017183|        null|
|2017-04-01|    150222596|      23841|   30485|          3|  09:05:00|10:35:00|          2|SI Apartment HSR ...|1491018557|  1491017183|
|2017-04-01|    150222596|      23841|   30485|          3|  09:05:00|10:35:00|          3|14th Main HSR Layout|1491018607|  1491018557|
|2017-04-01|    150222596|      23841|   30485|          3|  09:05:00|10:35:00|          4| Depot-25 HSR Layout|1491018688|  1491018607|
|2017-04-01|    150222596|      23841|   

In [102]:
# Instead of creating a persistent table using saveAsTable, make temp table and dump it as a hive table
vts_frame_all_stops_lagged_df.createOrReplaceTempView("temp_vts_frame_all_stops_lagged_df") 
sqlContext.sql("create table vts_full_bus_stops_apr17 as select * from temp_vts_frame_all_stops_lagged_df")
# sqlContext.sql("insert into vts_bus_stop_min_dist_apr17 select * from temp_VW_window_3_df")

DataFrame[]

In [103]:
sqlContext.sql("show tables").show(truncate=False)

+---------------------------+-----------+
|tableName                  |isTemporary|
+---------------------------+-----------+
|vts_365r                   |false      |
|vts_365r_distance          |false      |
|vts_365r_distance_1        |false      |
|vts_365r_filtered          |false      |
|vts_bus_stop_min_dist_apr17|false      |
|vts_bus_stop_min_dist_feb17|false      |
|vts_bus_stop_min_dist_jan17|false      |
|vts_bus_stop_min_dist_mar17|false      |
|vts_full_bus_stops_apr17   |false      |
|vts_full_bus_stops_feb17   |false      |
|vts_full_bus_stops_jan17   |false      |
|vts_full_bus_stops_mar17   |false      |
|vts_jun2017                |false      |
|vts_volvo_2017             |false      |
|vts_volvo_oct2017          |false      |
|vts_waybill_volvo_apr2017  |false      |
|vts_waybill_volvo_feb2017  |false      |
|vts_waybill_volvo_jan2017  |false      |
|vts_waybill_volvo_jun2017  |false      |
|vts_waybill_volvo_mar2017  |false      |
+---------------------------+-----

In [104]:
vts_full_bus_stops_one_month_df = sqlContext.sql("select * from vts_full_bus_stops_apr17")
vts_full_bus_stops_one_month_count = vts_full_bus_stops_one_month_df.count()
vts_full_bus_stops_one_month_count

2972514

In [105]:
vts_full_bus_stops_one_month_df.orderBy("duty_dt").groupBy("duty_dt").count().show(31)

+----------+------+
|   duty_dt| count|
+----------+------+
|2017-04-01| 80902|
|2017-04-02|101347|
|2017-04-03|117461|
|2017-04-04|111977|
|2017-04-05|101389|
|2017-04-06|108203|
|2017-04-07|101598|
|2017-04-08| 56678|
|2017-04-09| 99366|
|2017-04-10|113869|
|2017-04-11|112616|
|2017-04-12|115928|
|2017-04-13|110750|
|2017-04-14|111433|
|2017-04-15| 70354|
|2017-04-16| 91455|
|2017-04-17|110440|
|2017-04-18|109459|
|2017-04-19|103662|
|2017-04-20|107101|
|2017-04-21|119129|
|2017-04-22| 74183|
|2017-04-23| 94480|
|2017-04-24|113709|
|2017-04-25|118863|
|2017-04-26|109062|
|2017-04-27| 99037|
|2017-04-28| 90916|
|2017-04-29| 67115|
|2017-04-30| 50032|
+----------+------+

